이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [1]:
!pip install pyspark==3.3.1 py4j==0.10.9.5 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 9.6 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=ab316fa9de4d46b25066dfe942c5c8dce71a511b10beeec1507edf12ac6dab90
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


**Spark Session**을 하나 만든다

In [1]:
from pyspark.sql import *
from pyspark.sql.functions import *

if __name__ == "__main__":
    spark = SparkSession \
        .builder \
        .appName("Spark Writing Demo") \
        .master("spark://spark-master:7077") \
        .config("spark.executor.memory", "512m")\
        .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.3.1") \
        .getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2b63a418-feee-49b1-b37d-cdc0f4345dcc;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.3.1 in central
	found org.tukaani#xz;1.8 in central
	found org.spark-project.spark#unused;1.0.0 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/spark-avro_2.12/3.3.1/spark-avro_2.12-3.3.1.jar ...
	[SUCCESSFUL ] org.apache.spark#spark-avro_2.12;3.3.1!spark-avro_2.12.jar (413ms)
downloading https://repo1.maven.org/maven2/org/tukaani/xz/1.8/xz-1.8.jar ...
	[SUCCESSFUL ] org.tukaani#xz;1.8!xz.jar (290ms)
downloading https://repo1.maven.org/maven2/org/spark-project/spark/unused/1.0.0/unused-1.0.0.jar ...
	[SUCCESSFUL ] org.spark-project.spark#unused;1.0.0!unused.jar (265ms)
:: resolution report :: resolve 5044ms :: artifacts dl 973ms
	:: modules in use:


23/10/04 04:50:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
!wget https://pyspark-test-sj.s3.us-west-2.amazonaws.com/appl_stock.csv

--2023-09-25 11:51:47--  https://pyspark-test-sj.s3.us-west-2.amazonaws.com/appl_stock.csv
Resolving pyspark-test-sj.s3.us-west-2.amazonaws.com (pyspark-test-sj.s3.us-west-2.amazonaws.com)... 52.92.160.58, 3.5.79.16, 3.5.81.130, ...
Connecting to pyspark-test-sj.s3.us-west-2.amazonaws.com (pyspark-test-sj.s3.us-west-2.amazonaws.com)|52.92.160.58|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 143130 (140K) [text/csv]
Saving to: ‘appl_stock.csv’

appl_stock.csv      100%[===================>] 139.78K   506KB/s    in 0.3s    

2023-09-25 11:51:48 (506 KB/s) - ‘appl_stock.csv’ saved [143130/143130]



In [2]:
df = spark.read \
    .format("csv") \
    .load("appl_stock.csv")

In [3]:
print("Num Partitions before: " + str(df.rdd.getNumPartitions()))
df.groupBy(spark_partition_id()).count().show()

Num Partitions before: 1
+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   0| 1763|
+--------------------+-----+



In [4]:
df2 = df.repartition(4)
print("Num Partitions after: " + str(df2.rdd.getNumPartitions()))
df2.groupBy(spark_partition_id()).count().show()

Num Partitions after: 4
+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   0|  440|
|                   1|  441|
|                   2|  441|
|                   3|  441|
+--------------------+-----+



In [7]:
df3 = df2.coalesce(2)
print("Num Partitions after: " + str(df3.rdd.getNumPartitions()))
df3.groupBy(spark_partition_id()).count().show()

Num Partitions after: 2
+--------------------+-----+
|SPARK_PARTITION_ID()|count|
+--------------------+-----+
|                   0|  881|
|                   1|  882|
+--------------------+-----+



In [6]:
df.write \
    .format("avro") \
    .mode("overwrite") \
    .option("path", "dataOutput/avro/") \
    .save()

In [8]:
df2.write \
    .format("parquet") \
    .mode("overwrite") \
    .option("path", "dataOutput/parquet/") \
    .save()

In [9]:
df3.write \
    .format("json") \
    .mode("overwrite") \
    .option("path", "dataOutput/json/") \
    .save()

In [10]:
!ls -tl dataOutput/

total 0
drwxr-xr-x  8 root root 256 Sep 25 11:52 json
drwxr-xr-x 12 root root 384 Sep 25 11:52 parquet
drwxr-xr-x  6 root root 192 Sep 25 11:52 avro


In [11]:
!ls -tl dataOutput/parquet/

total 112
-rw-r--r-- 1 root root     0 Sep 25 11:52 _SUCCESS
-rw-r--r-- 1 root root 28024 Sep 25 11:52 part-00003-6712411b-9177-4c33-b678-ccd47c466483-c000.snappy.parquet
-rw-r--r-- 1 root root 28069 Sep 25 11:52 part-00002-6712411b-9177-4c33-b678-ccd47c466483-c000.snappy.parquet
-rw-r--r-- 1 root root 27787 Sep 25 11:52 part-00001-6712411b-9177-4c33-b678-ccd47c466483-c000.snappy.parquet
-rw-r--r-- 1 root root 27833 Sep 25 11:52 part-00000-6712411b-9177-4c33-b678-ccd47c466483-c000.snappy.parquet


In [12]:
!ls -tl dataOutput/avro/

total 88
-rw-r--r-- 1 root root     0 Sep 25 11:52 _SUCCESS
-rw-r--r-- 1 root root 89029 Sep 25 11:52 part-00000-7f68875e-5c21-4f0a-b800-cfe0a4cc9522-c000.avro


In [13]:
!ls -tl dataOutput/json/

total 240
-rw-r--r-- 1 root root      0 Sep 25 11:52 _SUCCESS
-rw-r--r-- 1 root root 122814 Sep 25 11:52 part-00001-392c6f31-d9fd-47a9-b775-e535636f4367-c000.json
-rw-r--r-- 1 root root 122570 Sep 25 11:52 part-00000-392c6f31-d9fd-47a9-b775-e535636f4367-c000.json
23/09/25 11:55:30 WARN JavaUtils: Attempt to delete using native Unix OS command failed for path = /tmp/blockmgr-e1a2898a-1080-48c3-b8eb-48ca233651e2. Falling back to Java IO way
java.io.IOException: Failed to delete: /tmp/blockmgr-e1a2898a-1080-48c3-b8eb-48ca233651e2
	at org.apache.spark.network.util.JavaUtils.deleteRecursivelyUsingUnixNative(JavaUtils.java:171)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:110)
	at org.apache.spark.network.util.JavaUtils.deleteRecursively(JavaUtils.java:91)
	at org.apache.spark.util.Utils$.deleteRecursively(Utils.scala:1206)
	at org.apache.spark.storage.DiskBlockManager.$anonfun$doStop$1(DiskBlockManager.scala:374)
	at org.apache.spark.storage.DiskBlockManager.